## Predicting Customer Churn in Telecom: A Classification Model Approach


### Hypothesis (H₁):
**"Payment methods and age of the customer, have a significant impact on the likelihood of churn in the telecommunication company."**

### Null Hypothesis (H₀):
**"Payment methods and age of the customer, do not have a significant impact on the likelihood of churn in the telecommunication company."**



Questions that would guide the analysis and exploration of this customer churn classification model project:

1. **Which customer demographics and service-related features (e.g., contract type, payment method, monthly charges) have the highest correlation with customer churn?**
  
2. **What is the impact of customer tenure on the likelihood of churn, and do long-term customers have a lower probability of leaving the telecom service?**

3. **How does the type of contract (monthly vs. yearly) affect churn behavior, and is there a significant difference in churn rates between different contract types?**

4. **Can targeted interventions based on the model's predictions reduce churn, and which customer segments should be prioritized for retention efforts?**
  



In [1]:
# installing pyodbc and python-dotenv
#%pip install pyodbc
#%pip install python-dotenv

In [2]:
#installing necessary packages.
import pyodbc
from dotenv import dotenv_values
import pandas as pd
import warnings


warnings.filterwarnings('ignore')

In [3]:
# Load environment variables from .env file into a dictionary
environment_variables = dotenv_values('.env')

# Get the values for the credentials you set in the '.env' file
server = environment_variables.get("SERVER")
database = environment_variables.get("DATABASE")
username = environment_variables.get("USERNAME")
password = environment_variables.get("PASSWORD")


In [4]:
# Create a connection string
connection_string = f"DRIVER={{SQL Server}};SERVER={server};DATABASE={database};UID={username};PWD={password};MARS_Connection=yes;MinProtocolVersion=TLSv1.2;"


In [5]:
# Use the connect method of the pyodbc library and pass in the connection string.
# This will connect to the server and might take a few seconds to be complete. 
# Check your internet connection if it takes more time than necessary

connection = pyodbc.connect(connection_string)


In [6]:
# Now the sql query to get the data is what what you see below. 
# Note that you will not have permissions to insert delete or update this database table. 

query = "SELECT * FROM LP2_Telco_churn_first_3000"

data_1 = pd.read_sql(query, connection)

In [7]:
data_1.head()

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,False,True,False,1,False,None,DSL,False,...,False,False,False,False,Month-to-month,True,Electronic check,29.850000,29.850000,False
1,5575-GNVDE,Male,False,False,False,34,True,False,DSL,True,...,True,False,False,False,One year,False,Mailed check,56.950001,1889.500000,False
2,3668-QPYBK,Male,False,False,False,2,True,False,DSL,True,...,False,False,False,False,Month-to-month,True,Mailed check,53.849998,108.150002,True
3,7795-CFOCW,Male,False,False,False,45,False,None,DSL,True,...,True,True,False,False,One year,False,Bank transfer (automatic),42.299999,1840.750000,False
4,9237-HQITU,Female,False,False,False,2,True,False,Fiber optic,False,...,False,False,False,False,Month-to-month,True,Electronic check,70.699997,151.649994,True


In [8]:
data_1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3000 entries, 0 to 2999
Data columns (total 21 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   customerID        3000 non-null   object 
 1   gender            3000 non-null   object 
 2   SeniorCitizen     3000 non-null   bool   
 3   Partner           3000 non-null   bool   
 4   Dependents        3000 non-null   bool   
 5   tenure            3000 non-null   int64  
 6   PhoneService      3000 non-null   bool   
 7   MultipleLines     2731 non-null   object 
 8   InternetService   3000 non-null   object 
 9   OnlineSecurity    2349 non-null   object 
 10  OnlineBackup      2349 non-null   object 
 11  DeviceProtection  2349 non-null   object 
 12  TechSupport       2349 non-null   object 
 13  StreamingTV       2349 non-null   object 
 14  StreamingMovies   2349 non-null   object 
 15  Contract          3000 non-null   object 
 16  PaperlessBilling  3000 non-null   bool   


In [9]:
data_1["gender"].nunique()

2

In [10]:
data_1["gender"].unique()

array(['Female', 'Male'], dtype=object)

In [11]:
data_1['SeniorCitizen'].unique()

array([False,  True])

In [12]:
data_1['Partner'].unique()

array([ True, False])

In [13]:
data_1['Dependents'].unique()

array([False,  True])

In [14]:
data_1['tenure'].unique()

array([ 1, 34,  2, 45,  8, 22, 10, 28, 62, 13, 16, 58, 49, 25, 69, 52, 71,
       21, 12, 30, 47, 72, 17, 27,  5, 46, 11, 70, 63, 43, 15, 60, 18, 66,
        9,  3, 31, 50, 64, 56,  7, 42, 35, 48, 29, 65, 38, 68, 32, 55, 37,
       36, 41,  6,  4, 33, 67, 23, 57, 61, 14, 20, 53, 40, 59, 24, 44, 19,
       54, 51, 26,  0, 39], dtype=int64)

In [15]:
data_1['PhoneService'].unique()

array([False,  True])

In [16]:
data_1['MultipleLines'].unique()

array([None, False, True], dtype=object)

In [17]:
data_1['MultipleLines'].ffill(inplace=True)

In [18]:
data_1['MultipleLines'].fillna(False,inplace=True)

In [19]:
data_1['MultipleLines'].value_counts()

MultipleLines
False    1578
True     1422
Name: count, dtype: int64

In [20]:
data_1['InternetService'].unique()

array(['DSL', 'Fiber optic', 'No'], dtype=object)

In [21]:
data_1['OnlineSecurity'].unique()

array([False, True, None], dtype=object)

In [22]:
data_1['OnlineSecurity'].bfill(inplace=True)

In [23]:
data_1['OnlineSecurity'].fillna(True,inplace=True)

In [24]:
data_1['OnlineBackup'].unique()

array([True, False, None], dtype=object)

In [25]:
data_1['OnlineBackup'].fillna(True,inplace=True)

In [26]:
data_1['TechSupport'].unique()

array([False, True, None], dtype=object)

In [27]:
data_1['TechSupport'].fillna(False,inplace=True)

In [28]:
data_1['DeviceProtection'].unique()

array([False, True, None], dtype=object)

In [29]:
data_1['DeviceProtection'].fillna(True,inplace=True)

In [30]:
data_1['StreamingTV'].unique()

array([False, True, None], dtype=object)

In [31]:
data_1['StreamingTV'].fillna(False,inplace=True)

In [32]:
data_1['StreamingMovies'].unique()

array([False, True, None], dtype=object)

In [33]:
data_1['StreamingMovies'].fillna(True,inplace=True)

In [34]:
data_1['Contract'].unique()

array(['Month-to-month', 'One year', 'Two year'], dtype=object)

In [35]:
data_1['PaperlessBilling'].unique()

array([ True, False])

In [36]:
data_1['PaymentMethod'].unique()

array(['Electronic check', 'Mailed check', 'Bank transfer (automatic)',
       'Credit card (automatic)'], dtype=object)

In [37]:
data_1['MonthlyCharges'].unique()

array([29.85000038, 56.95000076, 53.84999847, ..., 33.90000153,
       34.        , 38.59999847])

In [38]:
data_1['TotalCharges'].unique()

array([  29.85000038, 1889.5       ,  108.15000153, ..., 6143.14990234,
        144.80000305,  414.95001221])

In [39]:
data_1['TotalCharges'].fillna(data_1['TotalCharges'].mean(),inplace=True)

In [40]:
data_1['Churn'].unique()

array([False, True, None], dtype=object)

In [41]:
data_1['Churn'].fillna(False,inplace=True)

In [42]:
data_2 = pd.read_csv('Lp2_Telco-churn-second-2000.csv')

data_2.head()

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,5600-PDUJF,Male,0,No,No,6,Yes,No,DSL,No,...,No,Yes,No,No,Month-to-month,Yes,Credit card (automatic),49.50,312.7,No
1,8292-TYSPY,Male,0,No,No,19,Yes,No,DSL,No,...,Yes,Yes,No,No,Month-to-month,Yes,Credit card (automatic),55.00,1046.5,Yes
2,0567-XRHCU,Female,0,Yes,Yes,69,No,No phone service,DSL,Yes,...,Yes,No,No,Yes,Two year,Yes,Credit card (automatic),43.95,2960.1,No
3,1867-BDVFH,Male,0,Yes,Yes,11,Yes,Yes,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,74.35,834.2,Yes
4,2067-QYTCF,Female,0,Yes,No,64,Yes,Yes,Fiber optic,No,...,Yes,Yes,Yes,Yes,Month-to-month,Yes,Electronic check,111.15,6953.4,No


In [43]:
charges = data_2["TotalCharges"].to_list()
charges

['312.7',
 '1046.5',
 '2960.1',
 '834.2',
 '6953.4',
 '4134.85',
 '899.8',
 '541.5',
 '116.85',
 '7942.15',
 '5321.25',
 '4156.8',
 '223.6',
 '768.45',
 '3765.05',
 '2157.5',
 '108.05',
 '1391.65',
 '1379.8',
 '1273.3',
 '810.45',
 '1810.85',
 '7782.85',
 '70.3',
 '1797.1',
 '377.85',
 '445.3',
 '851.75',
 '7624.2',
 '355.1',
 '575.45',
 '906.85',
 '1057.55',
 '19.4',
 '1388.75',
 '1641.3',
 '1375.15',
 '152.7',
 '185.2',
 '195.05',
 '1826.7',
 '417.5',
 '344.5',
 '1660',
 '2180.55',
 '2835.9',
 '45.15',
 '2730.85',
 '5437.1',
 '20.2',
 '6981.35',
 '5794.45',
 '747.2',
 '1267.95',
 '3674.95',
 '1275.6',
 '5893.9',
 '724.65',
 '1536.75',
 '3615.6',
 '607.3',
 '4154.55',
 '2184.35',
 '1852.85',
 '77.15',
 '847.8',
 '2390.45',
 '1923.5',
 '1493.2',
 '338.1',
 '3030.6',
 '75.35',
 '2184.85',
 '1031.4',
 '921.3',
 '3875.4',
 '3124.5',
 '144.15',
 '6689',
 '1451.1',
 '6368.2',
 '3729.6',
 '350.1',
 '2847.2',
 '452.35',
 '1494.5',
 '930.45',
 '41.85',
 '1272.05',
 '475.1',
 '673.1',
 '208.7',

In [44]:
data_2["TotalCharges"]=pd.to_numeric(data_2["TotalCharges"], errors='coerce')
data_2["TotalCharges"]

0        312.70
1       1046.50
2       2960.10
3        834.20
4       6953.40
         ...   
2038    1990.50
2039    7362.90
2040     346.45
2041     306.60
2042    6844.50
Name: TotalCharges, Length: 2043, dtype: float64

In [45]:
data_2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2043 entries, 0 to 2042
Data columns (total 21 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   customerID        2043 non-null   object 
 1   gender            2043 non-null   object 
 2   SeniorCitizen     2043 non-null   int64  
 3   Partner           2043 non-null   object 
 4   Dependents        2043 non-null   object 
 5   tenure            2043 non-null   int64  
 6   PhoneService      2043 non-null   object 
 7   MultipleLines     2043 non-null   object 
 8   InternetService   2043 non-null   object 
 9   OnlineSecurity    2043 non-null   object 
 10  OnlineBackup      2043 non-null   object 
 11  DeviceProtection  2043 non-null   object 
 12  TechSupport       2043 non-null   object 
 13  StreamingTV       2043 non-null   object 
 14  StreamingMovies   2043 non-null   object 
 15  Contract          2043 non-null   object 
 16  PaperlessBilling  2043 non-null   object 


In [46]:
charges = data_2['TotalCharges'].to_list()
print(charges)

[312.7, 1046.5, 2960.1, 834.2, 6953.4, 4134.85, 899.8, 541.5, 116.85, 7942.15, 5321.25, 4156.8, 223.6, 768.45, 3765.05, 2157.5, 108.05, 1391.65, 1379.8, 1273.3, 810.45, 1810.85, 7782.85, 70.3, 1797.1, 377.85, 445.3, 851.75, 7624.2, 355.1, 575.45, 906.85, 1057.55, 19.4, 1388.75, 1641.3, 1375.15, 152.7, 185.2, 195.05, 1826.7, 417.5, 344.5, 1660.0, 2180.55, 2835.9, 45.15, 2730.85, 5437.1, 20.2, 6981.35, 5794.45, 747.2, 1267.95, 3674.95, 1275.6, 5893.9, 724.65, 1536.75, 3615.6, 607.3, 4154.55, 2184.35, 1852.85, 77.15, 847.8, 2390.45, 1923.5, 1493.2, 338.1, 3030.6, 75.35, 2184.85, 1031.4, 921.3, 3875.4, 3124.5, 144.15, 6689.0, 1451.1, 6368.2, 3729.6, 350.1, 2847.2, 452.35, 1494.5, 930.45, 41.85, 1272.05, 475.1, 673.1, 208.7, 150.75, 3199.0, 5844.65, 2462.6, 6263.8, 3403.4, 237.75, 221.35, 75.3, 1672.1, 7881.2, 2320.8, 370.5, 4062.2, 3043.7, 2723.4, 1081.25, 4097.05, 632.2, 4042.3, 164.85, 8166.8, 4113.7, 3047.15, 70.4, 4193.4, 3729.75, 399.45, 331.3, 1964.6, 1336.15, 1147.45, 486.05, 1078.9

In [47]:
train_data = pd.concat([data_1, data_2],ignore_index=True)
train_data

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,False,True,False,1,False,False,DSL,False,...,False,False,False,False,Month-to-month,True,Electronic check,29.850000,29.850000,False
1,5575-GNVDE,Male,False,False,False,34,True,False,DSL,True,...,True,False,False,False,One year,False,Mailed check,56.950001,1889.500000,False
2,3668-QPYBK,Male,False,False,False,2,True,False,DSL,True,...,False,False,False,False,Month-to-month,True,Mailed check,53.849998,108.150002,True
3,7795-CFOCW,Male,False,False,False,45,False,False,DSL,True,...,True,True,False,False,One year,False,Bank transfer (automatic),42.299999,1840.750000,False
4,9237-HQITU,Female,False,False,False,2,True,False,Fiber optic,False,...,False,False,False,False,Month-to-month,True,Electronic check,70.699997,151.649994,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5038,6840-RESVB,Male,0,Yes,Yes,24,Yes,Yes,DSL,Yes,...,Yes,Yes,Yes,Yes,One year,Yes,Mailed check,84.800000,1990.500000,No
5039,2234-XADUH,Female,0,Yes,Yes,72,Yes,Yes,Fiber optic,No,...,Yes,No,Yes,Yes,One year,Yes,Credit card (automatic),103.200000,7362.900000,No
5040,4801-JZAZL,Female,0,Yes,Yes,11,No,No phone service,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.600000,346.450000,No
5041,8361-LTMKD,Male,1,Yes,No,4,Yes,Yes,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Mailed check,74.400000,306.600000,Yes


In [48]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5043 entries, 0 to 5042
Data columns (total 21 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   customerID        5043 non-null   object 
 1   gender            5043 non-null   object 
 2   SeniorCitizen     5043 non-null   object 
 3   Partner           5043 non-null   object 
 4   Dependents        5043 non-null   object 
 5   tenure            5043 non-null   int64  
 6   PhoneService      5043 non-null   object 
 7   MultipleLines     5043 non-null   object 
 8   InternetService   5043 non-null   object 
 9   OnlineSecurity    5043 non-null   object 
 10  OnlineBackup      5043 non-null   object 
 11  DeviceProtection  5043 non-null   object 
 12  TechSupport       5043 non-null   object 
 13  StreamingTV       5043 non-null   object 
 14  StreamingMovies   5043 non-null   object 
 15  Contract          5043 non-null   object 
 16  PaperlessBilling  5043 non-null   object 


In [49]:
train_data['SeniorCitizen'].unique()

array([False, True], dtype=object)

In [50]:
train_data.replace({False:'No', True:'Yes'},inplace=True)

In [51]:
train_data['SeniorCitizen'].replace({False:0,True:1},inplace=True)

In [52]:
train_data["TotalCharges"].fillna(train_data["TotalCharges"].mean(),inplace=True)

In [53]:
train_data.to_csv('Train-set.csv',index=False)

In [54]:
from ydata_profiling import ProfileReport

In [55]:
profile = ProfileReport(train_data,title='Telecom-churn-5000')

In [56]:
profile.to_notebook_iframe()

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

### Data Preparation ###

In [57]:
train_data.head()

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,No,Yes,No,1,No,No,DSL,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.850000,29.850000,No
1,5575-GNVDE,Male,No,No,No,34,Yes,No,DSL,Yes,...,Yes,No,No,No,One year,No,Mailed check,56.950001,1889.500000,No
2,3668-QPYBK,Male,No,No,No,2,Yes,No,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Mailed check,53.849998,108.150002,Yes
3,7795-CFOCW,Male,No,No,No,45,No,No,DSL,Yes,...,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.299999,1840.750000,No
4,9237-HQITU,Female,No,No,No,2,Yes,No,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,70.699997,151.649994,Yes


In [58]:
train_data = train_data.drop(columns = ["customerID"])

In [59]:
# Create a copy of my data set

train_mod = train_data.copy()

In [60]:
# check for data imbalance

train_mod['Churn'].value_counts()


Churn
No     3707
Yes    1336
Name: count, dtype: int64

In [61]:
# Resampling

from sklearn.utils import resample

In [62]:
churn_major = train_mod[train_mod['Churn'] == 'No']
churn_minor = train_mod[train_mod['Churn'] == 'Yes']

minor_resampled = resample(churn_minor, replace = True, n_samples = len(churn_major), random_state = 42)

train_model = pd.concat([churn_major, minor_resampled])

train_model['Churn'].value_counts()


Churn
No     3707
Yes    3707
Name: count, dtype: int64

In [63]:
target_column = "Churn"

In [64]:
# Separate features (X) and target (y)


X = train_model.drop(columns=[target_column])
y = train_model[target_column]

In [65]:
num_cols = X.select_dtypes(include = ["int64", "float64"]).columns

cat_cols = X.select_dtypes(include = "object").columns


In [66]:
from sklearn.model_selection import train_test_split

In [67]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)


In [94]:
X_train

,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges
3334,Female,No,Yes,Yes,49,Yes,Yes,Fiber optic,No,Yes,Yes,Yes,Yes,Yes,Two year,Yes,Bank transfer (automatic),107.950000,5293.200000
3488,Male,Yes,Yes,No,45,Yes,Yes,Fiber optic,No,Yes,Yes,Yes,Yes,No,One year,Yes,Bank transfer (automatic),98.700000,4525.800000
1426,Male,Yes,Yes,No,71,Yes,Yes,Fiber optic,Yes,Yes,Yes,Yes,No,No,One year,Yes,Electronic check,92.000000,6585.200195
836,Male,No,Yes,Yes,62,Yes,No,DSL,Yes,No,Yes,Yes,No,Yes,One year,No,Credit card (automatic),71.400002,4487.299805
2734,Male,No,No,No,11,Yes,No,No,No,Yes,Yes,No,No,Yes,Month-to-month,No,Mailed check,20.250000,208.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1249,Male,No,No,No,1,Yes,No,DSL,No,No,No,No,No,No,Month-to-month,No,Mailed check,45.049999,45.049999
773,Female,No,Yes,No,5,Yes,No,Fiber optic,Yes,No,Yes,No,No,Yes,Month-to-month,Yes,Mailed check,88.900002,454.149994
4064,Male,No,No,No,1,No,No phone service,DSL,No,No,No,No,Yes,Yes,Month-to-month,Yes,Mailed check,45.300000,45.300000
1162,Female,No,No,No,67,Yes,No,DSL,Yes,No,Yes,Yes,No,Yes,One year,Yes,Bank transfer (automatic),72.349998,4991.500000


In [95]:
y_train

3334    1
3488    0
1426    0
836     0
2734    0
       ..
1249    1
773     1
4064    1
1162    0
1736    1
Name: Churn, Length: 5931, dtype: int32

In [68]:
y_train = y_train.map({"Yes":1, "No":0}).astype(int)
y_val = y_val.map({"Yes":1, "No":0}).astype(int)

In [69]:
print(num_cols)

Index(['tenure', 'MonthlyCharges', 'TotalCharges'], dtype='object')


In [70]:
print(cat_cols)

Index(['gender', 'SeniorCitizen', 'Partner', 'Dependents', 'PhoneService',
       'MultipleLines', 'InternetService', 'OnlineSecurity', 'OnlineBackup',
       'DeviceProtection', 'TechSupport', 'StreamingTV', 'StreamingMovies',
       'Contract', 'PaperlessBilling', 'PaymentMethod'],
      dtype='object')


In [71]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import QuantileTransformer, RobustScaler, OneHotEncoder, LabelEncoder
from sklearn. compose import ColumnTransformer


num_pipe = Pipeline(steps = [('scaler',RobustScaler()),('qt',QuantileTransformer(output_distribution = 'normal'))])

cat_pipe = Pipeline(steps = [('onehot', OneHotEncoder(handle_unknown = 'ignore', sparse_output = False))])

preprocessor = ColumnTransformer(transformers=[('numerical',num_pipe,num_cols),('categorical',cat_pipe,cat_cols)])

preprocessor

ColumnTransformer(transformers=[('numerical',
                                 Pipeline(steps=[('scaler', RobustScaler()),
                                                 ('qt',
                                                  QuantileTransformer(output_distribution='normal'))]),
                                 Index(['tenure', 'MonthlyCharges', 'TotalCharges'], dtype='object')),
                                ('categorical',
                                 Pipeline(steps=[('onehot',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse_output=False))]),
                                 Index(['gender', 'SeniorCitizen', 'Partner', 'Dependents', 'PhoneService',
       'MultipleLines', 'InternetService', 'OnlineSecurity', 'OnlineBackup',
       'DeviceProtection', 'TechSupport', 'StreamingTV', 'StreamingMovies',
       'Contract', 'PaperlessBilling', 'PaymentMethod'],
      dtype='object'))])

### Modeling and Evaluation ###

In [72]:
%pip install catboost


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: C:\Users\Akinremi Timothy\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [73]:
%pip install lightgbm

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: C:\Users\Akinremi Timothy\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [74]:
%pip install imbalanced-learn

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: C:\Users\Akinremi Timothy\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [75]:
# Model Selection

from sklearn.ensemble import AdaBoostClassifier, RandomForestClassifier, GradientBoostingClassifier 
from sklearn .linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn import svm
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from catboost import CatBoostClassifier
import lightgbm as lgb
from xgboost import XGBClassifier
from imblearn.over_sampling import SMOTE





In [76]:
from imblearn.pipeline import Pipeline as ImbPipeline

In [77]:
models = [
    ("KNN", KNeighborsClassifier(n_neighbors=10)),
    ("LR", LogisticRegression(max_iter=1000, random_state=42, verbose=0)),
    ("RF", RandomForestClassifier(random_state=42, n_estimators=100, max_depth=None)),
    ("SVC", SVC(probability=True, random_state=42, C=1.0, gamma="scale")),
    ("GB", GradientBoostingClassifier(random_state=42, n_estimators=100, learning_rate=0.1, max_depth=3)),
    ("XB", XGBClassifier(random_state=42, use_label_encoder=False, eval_metric="logloss", n_estimators=100)),
    ("DT", DecisionTreeClassifier(random_state=42,max_depth=None))
]


# Define Oversampling Technique

sampler = SMOTE(random_state=42)

pipelines = []

for model, classifier in models:
    pipeline = ImbPipeline(steps=[
        ("preprocessor", preprocessor),
        ("smote_sampler", sampler),
        ("model", classifier)
    ])

    pipelines.append((model,pipeline))

In [78]:
 # importing performance metrics

from sklearn.metrics import accuracy_score, f1_score, precision_score,recall_score, confusion_matrix

In [79]:
eval_result = {}

for model, classifier in models:

    pipeline = ImbPipeline(steps=[
        ("preprocessor", preprocessor),
        ("smote_sampler", sampler),
        ("model", classifier)
    ])

    pipeline.fit(X_train,y_train)

    X_val_processed = pipeline.named_steps['preprocessor'].transform(X_val)

    y_pred = pipeline.named_steps['model'].predict(X_val_processed)

    accuracy = accuracy_score(y_val, y_pred)
    f1 = f1_score(y_val, y_pred)
    precision = precision_score(y_val, y_pred)
    recall = recall_score(y_val, y_pred)
    conf_matrix = confusion_matrix(y_val,y_pred)



    eval_result[model] = {
        'Accuracy': accuracy,
        'F1_score': f1,
        'Precision': precision,
        'Recall': recall,
        'Confusion_matrix': conf_matrix,
        'Predictions': y_pred}


scores_df = pd.DataFrame(eval_result).transpose()
scores_dff = scores_df.sort_values(by = "F1_score", ascending = False)

print('Model Scores: ')
scores_dff 

Model Scores: 


,Accuracy,F1_score,Precision,Recall,Confusion_matrix,Predictions
RF,0.903574,0.907203,0.861899,0.957534,"[[641, 112], [31, 699]]","[0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, ..."
DT,0.870533,0.877707,0.820238,0.943836,"[[602, 151], [41, 689]]","[0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, ..."
XB,0.869858,0.87652,0.822329,0.938356,"[[605, 148], [45, 685]]","[0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, ..."
SVC,0.811868,0.823082,0.766234,0.889041,"[[555, 198], [81, 649]]","[0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
GB,0.79973,0.809737,0.760529,0.865753,"[[554, 199], [98, 632]]","[0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
LR,0.783547,0.793569,0.747879,0.845205,"[[545, 208], [113, 617]]","[0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
KNN,0.778827,0.78866,0.744526,0.838356,"[[543, 210], [118, 612]]","[0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, ..."


In [80]:
import matplotlib.pyplot as plt
import seaborn as sns

In [81]:
def confusion_matrix(conf_matrix,model):
    plt.figure(figsize=(5,4))
    sns.heatmap(conf_matrix, annot=True, fmt= 'd',cmap = 'Blues', xticklabels =['No','Yes'],yticklabels=['No','Yes'])
    plt.title(f'Confusion matrix for {model}')
    plt.xlabel('Predicted Label')
    plt.ylabel('Actual Label')
    plt.show()

for model in eval_result.keys():
    conf_matrix = eval_result[model]['Confusion_matrix']
    confusion_matrix(conf_matrix,model)

In [ ]:
from sklearn.metrics import classification_report

for  model, classifier in models:
    print(f'Classification report for {model}: ')
    print(classification_report(y_val,y_pred, target_names= ['No','Yes']))
    print("="*30+"\n")

Classification report for KNN: 
              precision    recall  f1-score   support

          No       0.94      0.80      0.86       753
         Yes       0.82      0.94      0.88       730

    accuracy                           0.87      1483
   macro avg       0.88      0.87      0.87      1483
weighted avg       0.88      0.87      0.87      1483


Classification report for LR: 
              precision    recall  f1-score   support

          No       0.94      0.80      0.86       753
         Yes       0.82      0.94      0.88       730

    accuracy                           0.87      1483
   macro avg       0.88      0.87      0.87      1483
weighted avg       0.88      0.87      0.87      1483


Classification report for RF: 
              precision    recall  f1-score   support

          No       0.94      0.80      0.86       753
         Yes       0.82      0.94      0.88       730

    accuracy                           0.87      1483
   macro avg       0.88      0.8

### Hyperparameter Tunning

In [120]:
from sklearn.model_selection import GridSearchCV
import numpy as np
from sklearn.metrics import confusion_matrix
# Define hyperparameter grids
param_grids = {
    "KNN": {"model__n_neighbors": [5, 10, 15, 20]},
    "LR": {"model__C": [0.1, 1, 10], "model__solver": ["lbfgs", "liblinear"]},
    "RF": {"model__n_estimators": [50, 100, 200], "model__max_depth": [None, 10, 20]},
    "SVC": {"model__C": [0.1, 1, 10], "model__gamma": ["scale", "auto"]},
    "GB": {"model__n_estimators": [50, 100, 200], "model__learning_rate": [0.01, 0.1, 0.2]},
    "XB": {"model__n_estimators": [50, 100, 200], "model__learning_rate": [0.01, 0.1, 0.2]},
    "DT": {"model__max_depth": [None, 10, 20], "model__min_samples_split": [2, 5, 10]}
}

# Define Oversampling Technique
sampler = SMOTE(random_state=42)

# Initialize results storage
tuned_eval_result = {}

# Perform hyperparameter tuning for each model
for model_name, classifier in models:
    print(f"Tuning hyperparameters for {model_name}...")

    pipeline = ImbPipeline(steps=[
        ("preprocessor", preprocessor),
        ("smote_sampler", sampler),
        ("model", classifier)
    ])

    # Define GridSearchCV
    param_grid = param_grids.get(model_name, {})
    grid_search = GridSearchCV(
        estimator=pipeline,
        param_grid=param_grid,
        scoring="f1",  # Use F1-score for optimization
        cv=5,  # 5-fold cross-validation
        verbose=1,
        n_jobs=-1
    )

    # Fit the GridSearchCV object
    grid_search.fit(X_train, y_train)

    # Best model and evaluation
    best_pipeline = grid_search.best_estimator_
    best_params = grid_search.best_params_

    # Evaluate the tuned model
    y_pred = best_pipeline.predict(X_val)
    accuracy = accuracy_score(y_val, y_pred)
    f1 = f1_score(y_val, y_pred)
    precision = precision_score(y_val, y_pred)
    recall = recall_score(y_val, y_pred)
    conf_matrix = confusion_matrix(y_val, y_pred)

    # Store results
    tuned_eval_result[model_name] = {
        "Best Parameters": best_params,
        "Accuracy": accuracy,
        "F1_score": f1,
        "Precision": precision,
        "Recall": recall,
        "Confusion_matrix": conf_matrix
    }

# Convert results to DataFrame
tuned_scores_df = pd.DataFrame(tuned_eval_result).transpose()
tuned_scores_df = tuned_scores_df.sort_values(by="F1_score", ascending=False)

print("Tuned Model Scores:")
print(tuned_scores_df)


Tuning hyperparameters for KNN...
Fitting 5 folds for each of 4 candidates, totalling 20 fits
Tuning hyperparameters for LR...
Fitting 5 folds for each of 6 candidates, totalling 30 fits
Tuning hyperparameters for RF...
Fitting 5 folds for each of 9 candidates, totalling 45 fits
Tuning hyperparameters for SVC...
Fitting 5 folds for each of 6 candidates, totalling 30 fits
Tuning hyperparameters for GB...
Fitting 5 folds for each of 9 candidates, totalling 45 fits
Tuning hyperparameters for XB...
Fitting 5 folds for each of 9 candidates, totalling 45 fits
Tuning hyperparameters for DT...
Fitting 5 folds for each of 9 candidates, totalling 45 fits
Tuned Model Scores:
                                       Best Parameters  Accuracy  F1_score  \
RF   {'model__max_depth': None, 'model__n_estimator...  0.903574  0.907203   
XB   {'model__learning_rate': 0.2, 'model__n_estima...   0.87323  0.880559   
DT   {'model__max_depth': None, 'model__min_samples...  0.870533  0.877707   
GB   {'model__l

In [97]:
data_3 =  pd.read_excel('Telco-churn-last-2000.xlsx')

data_3.head()

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges
0,7613-LLQFO,Male,0,No,No,12,Yes,Yes,Fiber optic,No,No,No,No,Yes,No,Month-to-month,Yes,Electronic check,84.45,1059.55
1,4568-TTZRT,Male,0,No,No,9,Yes,No,No,No internet service,No internet service,No internet service,No internet service,No internet service,No internet service,Month-to-month,No,Mailed check,20.40,181.8
2,9513-DXHDA,Male,0,No,No,27,Yes,No,DSL,Yes,No,Yes,Yes,Yes,Yes,One year,No,Electronic check,81.70,2212.55
3,2640-PMGFL,Male,0,No,Yes,27,Yes,Yes,Fiber optic,No,No,No,Yes,No,No,Month-to-month,Yes,Electronic check,79.50,2180.55
4,3801-HMYNL,Male,0,Yes,Yes,1,Yes,No,Fiber optic,No,No,No,No,Yes,Yes,Month-to-month,No,Mailed check,89.15,89.15


In [98]:
data_3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 20 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   customerID        2000 non-null   object 
 1   gender            2000 non-null   object 
 2   SeniorCitizen     2000 non-null   int64  
 3   Partner           2000 non-null   object 
 4   Dependents        2000 non-null   object 
 5   tenure            2000 non-null   int64  
 6   PhoneService      2000 non-null   object 
 7   MultipleLines     2000 non-null   object 
 8   InternetService   2000 non-null   object 
 9   OnlineSecurity    2000 non-null   object 
 10  OnlineBackup      2000 non-null   object 
 11  DeviceProtection  2000 non-null   object 
 12  TechSupport       2000 non-null   object 
 13  StreamingTV       2000 non-null   object 
 14  StreamingMovies   2000 non-null   object 
 15  Contract          2000 non-null   object 
 16  PaperlessBilling  2000 non-null   object 


In [99]:
test_data = data_3.copy()

In [102]:
test_data=test_data.drop(columns=['customerID'])

In [105]:
test_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 19 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   gender            2000 non-null   object 
 1   SeniorCitizen     2000 non-null   int64  
 2   Partner           2000 non-null   object 
 3   Dependents        2000 non-null   object 
 4   tenure            2000 non-null   int64  
 5   PhoneService      2000 non-null   object 
 6   MultipleLines     2000 non-null   object 
 7   InternetService   2000 non-null   object 
 8   OnlineSecurity    2000 non-null   object 
 9   OnlineBackup      2000 non-null   object 
 10  DeviceProtection  2000 non-null   object 
 11  TechSupport       2000 non-null   object 
 12  StreamingTV       2000 non-null   object 
 13  StreamingMovies   2000 non-null   object 
 14  Contract          2000 non-null   object 
 15  PaperlessBilling  2000 non-null   object 
 16  PaymentMethod     2000 non-null   object 


In [104]:
test_data["TotalCharges"]=pd.to_numeric(test_data["TotalCharges"], errors='coerce')
test_data["TotalCharges"]

0       1059.55
1        181.80
2       2212.55
3       2180.55
4         89.15
         ...   
1995     552.95
1996    3496.30
1997      93.70
1998    7053.35
1999     301.55
Name: TotalCharges, Length: 2000, dtype: float64

In [107]:
import numpy as np

In [108]:

for col in test_data.columns:
    print(f'column:{col},Type:{test_data[col].dtype}')
    #Check if applying np.isnan would fail
    try:
        np.isnan(test_data[col])
    except TypeError:
        print(f"np.isnan() not applicable to column: {col}")

column:gender,Type:object
np.isnan() not applicable to column: gender
column:SeniorCitizen,Type:int64
column:Partner,Type:object
np.isnan() not applicable to column: Partner
column:Dependents,Type:object
np.isnan() not applicable to column: Dependents
column:tenure,Type:int64
column:PhoneService,Type:object
np.isnan() not applicable to column: PhoneService
column:MultipleLines,Type:object
np.isnan() not applicable to column: MultipleLines
column:InternetService,Type:object
np.isnan() not applicable to column: InternetService
column:OnlineSecurity,Type:object
np.isnan() not applicable to column: OnlineSecurity
column:OnlineBackup,Type:object
np.isnan() not applicable to column: OnlineBackup
column:DeviceProtection,Type:object
np.isnan() not applicable to column: DeviceProtection
column:TechSupport,Type:object
np.isnan() not applicable to column: TechSupport
column:StreamingTV,Type:object
np.isnan() not applicable to column: StreamingTV
column:StreamingMovies,Type:object
np.isnan() not a

In [110]:
#1: Select only numeric columns
numeric_cols = test_data.select_dtypes(include = ['float64','int64'])

#2: Find rows where there are NaN values in numeric columns
nan_rows = numeric_cols.isna().any(axis=1)

#3: Use the boolean mask to filter the original DataFrame
rows_with_nan = test_data[nan_rows]

# Display the rows with NaN values
rows_with_nan

,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges
331,Male,0,Yes,Yes,0,Yes,No,No,No internet service,No internet service,No internet service,No internet service,No internet service,No internet service,Two year,No,Mailed check,19.85,NaN
826,Male,0,Yes,Yes,0,Yes,Yes,No,No internet service,No internet service,No internet service,No internet service,No internet service,No internet service,Two year,No,Mailed check,25.35,NaN
1380,Female,0,Yes,Yes,0,Yes,No,No,No internet service,No internet service,No internet service,No internet service,No internet service,No internet service,Two year,No,Mailed check,20.00,NaN


In [111]:
# Filling for Numerical - TotalCharges column

test_data['TotalCharges'].fillna(test_data['TotalCharges'].median(),inplace = True)

In [112]:
# Check if columns in test_data match those in the training dataset
print(test_data.columns)
print(X_train.columns)

Index(['gender', 'SeniorCitizen', 'Partner', 'Dependents', 'tenure',
       'PhoneService', 'MultipleLines', 'InternetService', 'OnlineSecurity',
       'OnlineBackup', 'DeviceProtection', 'TechSupport', 'StreamingTV',
       'StreamingMovies', 'Contract', 'PaperlessBilling', 'PaymentMethod',
       'MonthlyCharges', 'TotalCharges'],
      dtype='object')
Index(['gender', 'SeniorCitizen', 'Partner', 'Dependents', 'tenure',
       'PhoneService', 'MultipleLines', 'InternetService', 'OnlineSecurity',
       'OnlineBackup', 'DeviceProtection', 'TechSupport', 'StreamingTV',
       'StreamingMovies', 'Contract', 'PaperlessBilling', 'PaymentMethod',
       'MonthlyCharges', 'TotalCharges'],
      dtype='object')


In [113]:
# Using the test_data
test_data['SeniorCitizen'] = test_data['SeniorCitizen'].replace({0:'No',1:'Yes'},inplace = True)

In [114]:
print("Training features: ",preprocessor.get_feature_names_out())
print("Test set features: ",test_data.columns)

Training features:  ['numerical__tenure' 'numerical__MonthlyCharges' 'numerical__TotalCharges'
 'categorical__gender_Female' 'categorical__gender_Male'
 'categorical__SeniorCitizen_No' 'categorical__SeniorCitizen_Yes'
 'categorical__Partner_No' 'categorical__Partner_Yes'
 'categorical__Dependents_No' 'categorical__Dependents_Yes'
 'categorical__PhoneService_No' 'categorical__PhoneService_Yes'
 'categorical__MultipleLines_No'
 'categorical__MultipleLines_No phone service'
 'categorical__MultipleLines_Yes' 'categorical__InternetService_DSL'
 'categorical__InternetService_Fiber optic'
 'categorical__InternetService_No' 'categorical__OnlineSecurity_No'
 'categorical__OnlineSecurity_No internet service'
 'categorical__OnlineSecurity_Yes' 'categorical__OnlineBackup_No'
 'categorical__OnlineBackup_No internet service'
 'categorical__OnlineBackup_Yes' 'categorical__DeviceProtection_No'
 'categorical__DeviceProtection_No internet service'
 'categorical__DeviceProtection_Yes' 'categorical__TechS

In [130]:
best_estimators = {}
y_test = {}

for model in ['RF','XB']:
    if model in grid_search:
        best_estimators[model]=grid_search[model].best_estimators_
    else:
        print(f"{model} is not found")

TypeError: argument of type 'GridSearchCV' is not iterable

In [ ]:
for model, model in best_estimators.items():
    y_test[model] = model.predict(test_data)

In [ ]:
# print the predictions for each model
for model, predictions in y_test.items():
    print(predictions)

In [ ]:
from sklearn.metrics import roc_auc_score

# Initialize results dictionary
results_test = {}

# loop through grid_searches_tuned items
for model, grid_search_tuned in grid_searches_tuned.items():
    # Ensure only RF and XB models are processed
    if model not in ['RF','XB']:
        continue
    # predict on evaluation data
    y_test_pred = grid_search_tuned.predict(test_data)
    #calculate other matrices
    accuracy = accuracy_score(predictions, y_test_pred)
    f1 = f1_score(predictions, y_test_pred)
    precision = precision_score(predictions, y_test_pred)
    recall = recall_score(predictions, y_test_pred)
    conf_matrix = confusion_matrix(predictions, y_test_pred)
    y_test_prob = grid_search_tuned.predict_proba(test_data)[:,1]
    roc_auc = roc_auc_score(predictions, y_test_prob)

    # Store the results in eval_results_tuned dictionary
    results_test[model] = {
        "accuracy": accuracy,
        "precision": precision,
        "recall": recall,
        "f1_score": f1,
        "ROC AUC Score": roc_auc,
        "predictions": y_pred,
        "y_prob": y_prob 
    }


    # Convert results to DataFrame
scores_test = pd.DataFrame(results_test).transpose()
scores_test_sorted = scores_test.sort_values(by="ROC AUC Score", ascending = False)

#Print sorted evaluation scores
print("Model Score: ")
score_test_sorted